need to use colab for gpus

1. Setup and Load Data

1.1 Install Dependencies and Setup

In [1]:
import os
import cv2
import imghdr

1.3 Load Data

In [2]:
import numpy as np
from matplotlib import pyplot as plt

2. Preprocessing

In [3]:
import numpy as np
import torchvision.models.segmentation
import torch
import torchvision.transforms as T
from PIL import Image
import scipy.io

In [4]:
import tqdm

image_list = []
theta_list = []

directory = 'trainingdataimages'
 
# iterate over files in
# that directory
for filename in tqdm.tqdm(os.listdir(directory)):
    print(filename)
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        # Opens a image in RGB mode
        image = Image.open(f)
        data = np.asarray(image)
        matfile = 'trainingdatavalues/' + filename[:len(filename) - 4] + '.mat'
        mat = scipy.io.loadmat(matfile)
        image_list.append(data)
        theta_list.append(mat['th0'][0])

print(image_list[0].shape)
print(theta_list[0])
image_list = torch.tensor(image_list)
theta_list = torch.tensor(theta_list)

image_list.shape

 24%|██▍       | 19/80 [00:00<00:00, 96.13it/s]

gentrain_001.png
gentrain_002.png
gentrain_003.png
gentrain_004.png
gentrain_005.png
gentrain_006.png
gentrain_007.png
gentrain_008.png
gentrain_009.png
gentrain_010.png
gentrain_011.png
gentrain_012.png
gentrain_013.png
gentrain_014.png
gentrain_015.png
gentrain_016.png
gentrain_017.png
gentrain_018.png
gentrain_019.png


 40%|████      | 32/80 [00:00<00:00, 107.97it/s]

gentrain_020.png
gentrain_021.png
gentrain_022.png
gentrain_023.png
gentrain_024.png
gentrain_025.png
gentrain_026.png
gentrain_027.png
gentrain_028.png
gentrain_029.png
gentrain_030.png
gentrain_031.png
gentrain_032.png
gentrain_033.png
gentrain_034.png
gentrain_035.png
gentrain_036.png
gentrain_037.png
gentrain_038.png
gentrain_039.png
gentrain_040.png
gentrain_041.png
gentrain_042.png
gentrain_043.png
gentrain_044.png
gentrain_045.png


 71%|███████▏  | 57/80 [00:00<00:00, 109.83it/s]

gentrain_046.png
gentrain_047.png
gentrain_048.png
gentrain_049.png
gentrain_050.png
gentrain_051.png
gentrain_052.png
gentrain_053.png
gentrain_054.png
gentrain_055.png
gentrain_056.png
gentrain_057.png
gentrain_058.png
gentrain_059.png
gentrain_060.png
gentrain_061.png
gentrain_062.png
gentrain_063.png
gentrain_064.png
gentrain_065.png
gentrain_066.png
gentrain_067.png


100%|██████████| 80/80 [00:00<00:00, 110.03it/s]

gentrain_068.png
gentrain_069.png
gentrain_070.png
gentrain_071.png
gentrain_072.png
gentrain_073.png
gentrain_074.png
gentrain_075.png
gentrain_076.png
gentrain_077.png
gentrain_078.png
gentrain_079.png
gentrain_080.png
(526, 526, 3)
[1.6e+06 2.5e+00 2.0e+04]



C:\Users\benli\AppData\Local\Temp\ipykernel_37396\4196441949.py:25: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  image_list = torch.tensor(image_list)


torch.Size([80, 526, 526, 3])

In [5]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') # Set device GPU or CPU where the training will take place
Net = torchvision.models.resnet18() # Load net
Net.fc = torch.nn.Linear(in_features=512, out_features=1, bias=True) # Change final layer to predict one value
Net = Net.to(device)
optimizer = torch.optim.Adam(params=Net.parameters(),lr=Learning_Rate) # Create adam optimizer

print(Net)

NameError: name 'Learning_Rate' is not defined

In [ ]:
AverageLoss=np.zeros([50]) # Save average loss for display
for itr in range(2000): # Training loop
    images,theta0=LoadBatch() # Load taining batch
    images=torch.autograd.Variable(images,requires_grad=False).to(device) 
    theta0 = torch.autograd.Variable(theta0, requires_grad=False).to(device)
 
    pred=Net(images) # make prediction
    Net.zero_grad()
    Loss=torch.abs(pred-theta0).mean()
    Loss.backward() # Backpropogate loss
    optimizer.step() # Apply gradient descent change to weight
    AverageLoss[itr%50]=Loss.data.cpu().numpy() # Save loss average
    print(itr,") Loss=",Loss.data.cpu().numpy(),'AverageLoss',AverageLoss.mean()) # Display loss
    if itr % 200 == 0: # Save model
        print("Saving Model" +str(itr) + ".torch") #Save model weight
        torch.save(Net.state_dict(),   str(itr) + ".torch")

0 ) Loss= 1599999.8 AverageLoss 31999.995
Saving Model0.torch
1 ) Loss= 1599999.8 AverageLoss 63999.99
2 ) Loss= 1599999.6 AverageLoss 95999.9825
3 ) Loss= 1599999.6 AverageLoss 127999.975
4 ) Loss= 1599999.5 AverageLoss 159999.965
5 ) Loss= 1599999.5 AverageLoss 191999.955
6 ) Loss= 1599999.4 AverageLoss 223999.9425
7 ) Loss= 1599999.4 AverageLoss 255999.93
8 ) Loss= 1599999.2 AverageLoss 287999.915
9 ) Loss= 1599999.2 AverageLoss 319999.9


KeyboardInterrupt: 